In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import time
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [16]:
df=pd.read_excel(r"../dataset/data.xlsx",sheet_name="Sheet1")
df

,Number of Storeys(NS),Height of Structure(HS),Number of Spans(HSP),Length of Spans (LS),Opening percentage (OP),Masonry wall Stiffeness Et (MS),Period (P)
0,1,3,2,3.0,0,2.25,0.092970
1,1,3,2,3.0,100,2.25,0.159750
2,1,3,2,4.5,0,2.25,0.094770
3,1,3,2,4.5,100,2.25,0.156589
4,1,3,2,6.0,0,2.25,0.106350
...,...,...,...,...,...,...,...
4021,22,66,6,6.0,50,25.00,1.628000
4022,22,66,6,6.0,75,25.00,2.714000
4023,22,66,6,6.0,100,25.00,2.879000
4024,22,66,6,7.5,0,25.00,0.602680


In [17]:
df.shape

(4026, 7)

In [18]:
df.head()

,Number of Storeys(NS),Height of Structure(HS),Number of Spans(HSP),Length of Spans (LS),Opening percentage (OP),Masonry wall Stiffeness Et (MS),Period (P)
0,1,3,2,3.0,0,2.25,0.092970
1,1,3,2,3.0,100,2.25,0.159750
2,1,3,2,4.5,0,2.25,0.094770
3,1,3,2,4.5,100,2.25,0.156589
4,1,3,2,6.0,0,2.25,0.106350


In [19]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df)

normalized_df = pd.DataFrame(normalized_data, columns=df.columns)

print(normalized_df)

      Number of Storeys(NS)  Height of Structure(HS)  Number of Spans(HSP)  \
0                       0.0                      0.0                   0.0   
1                       0.0                      0.0                   0.0   
2                       0.0                      0.0                   0.0   
3                       0.0                      0.0                   0.0   
4                       0.0                      0.0                   0.0   
...                     ...                      ...                   ...   
4021                    1.0                      1.0                   1.0   
4022                    1.0                      1.0                   1.0   
4023                    1.0                      1.0                   1.0   
4024                    1.0                      1.0                   1.0   
4025                    1.0                      1.0                   1.0   

      Length of Spans (LS)  Opening percentage (OP)  \
0       

In [20]:
normalized_df.keys()

Index(['Number of Storeys(NS)', 'Height of Structure(HS)',
       'Number of Spans(HSP)', 'Length of Spans (LS)',
       'Opening percentage (OP)', 'Masonry wall Stiffeness Et (MS)',
       'Period (P)'],
      dtype='object')

In [21]:
X=normalized_df.drop("Period (P)" ,axis=1)

In [22]:
y=normalized_df["Period (P)"]

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=99)

# LightGBM

In [24]:
import lightgbm as lgb

In [25]:
from sklearn.model_selection import KFold,cross_validate
from bayes_opt import BayesianOptimization
from hyperopt import hp,fmin,tpe,Trials,partial
from hyperopt.early_stop import no_progress_loss
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold
import optuna.integration.lightgbm as oplg

In [26]:
def bayesopt_objective(n_estimators,max_depth,num_leaves,learning_rate):
    model= lgb.LGBMRegressor(n_estimators=int(n_estimators)
              ,max_depth=int(max_depth)
              ,num_leaves=int(num_leaves)
              ,learning_rate=learning_rate
              )
    cv=KFold(n_splits=10,shuffle=True,random_state=99)
    validate_loss=cross_validate(model,X_train,y_train
                                   ,cv=cv
                                   ,scoring='neg_root_mean_squared_error'
                                   ,n_jobs=-1
                                   ,error_score='raise')
    
    return np.mean(abs(validate_loss['test_score']))

In [27]:
def param_bayes_opt(init_points,n_iter):
    opt=BayesianOptimization(bayesopt_objective
                             ,param_grid_simple
                             ,random_state=99)
    
        opt.maximize(init_points=init_points 
                 ,n_iter=n_iter 
                )
    
    params_best=opt.max['params'] 
    score_best=opt.max['target']
    
   
    print("\n","best params: ", params_best,
          "\n","best cvscore: ", score_best)
    
    return params_best,score_best

In [28]:
def bayes_opt_validation(params_best):
    model= lgb.LGBMRegressor(n_estimators=int(params_best['n_estimators'])
            ,max_depth=int(params_best['max_depth'])
            ,num_leaves=int(params_best['num_leaves'])
            ,learning_rate=params_best['learning_rate']
           )
    cv=KFold(n_splits=10,shuffle=True,random_state=99)
    validate_loss=cross_validate(model,X_train,y_train
                                   ,cv=cv
                                   ,scoring='neg_root_mean_squared_error'
                                   ,n_jobs=-1
                                  )
    
    return np.mean(abs(validate_loss['test_score']))

In [29]:
# Start time
start_time = time.time()

param_grid_simple={'n_estimators':(800,5000)
                  ,'max_depth':(2,10)
                  ,'num_leaves':(10,50)
                   ,'learning_rate':( 0.001,0.1)
                  }

params_best,score_best=param_bayes_opt(20,100)

params_best 
score_best 
validation_score=bayes_opt_validation(params_best)
validation_score
# End time
end_time = time.time()
# Calculate total time
total_time = end_time - start_time
print("Total time taken: ", total_time)

|   iter    |  target   | learni... | max_depth | n_esti... | num_le... |
-------------------------------------------------------------------------
| 1         | 0.003874  | 0.06756   | 5.905     | 4.267e+03 | 11.26     |
| 2         | 0.004197  | 0.081     | 6.525     | 2.05e+03  | 11.87     |
| 3         | 0.02343   | 0.09907   | 2.055     | 4.033e+03 | 39.87     |
| 4         | 0.004068  | 0.03837   | 5.953     | 4.702e+03 | 25.82     |
| 5         | 0.004241  | 0.09742   | 6.195     | 1.193e+03 | 42.53     |
| 6         | 0.005329  | 0.02196   | 6.435     | 2.028e+03 | 42.65     |
| 7         | 0.006887  | 0.08298   | 3.773     | 3.508e+03 | 13.81     |
| 8         | 0.02642   | 0.04175   | 2.775     | 1.405e+03 | 18.49     |
| 9         | 0.02608   | 0.04819   | 2.621     | 1.787e+03 | 10.26     |
| 10        | 0.004154  | 0.08997   | 6.418     | 1.504e+03 | 47.16     |
| 11        | 0.0259    | 0.05467   | 2.334     | 3.004e+03 | 35.61     |
| 12        | 0.004077  | 0.08027   | 

| 67        | 0.003764  | 0.07637   | 8.446     | 3.672e+03 | 46.9      |
| 68        | 0.004118  | 0.08249   | 8.731     | 1.435e+03 | 29.33     |
| 69        | 0.007099  | 0.05025   | 3.563     | 4.717e+03 | 33.29     |
| 70        | 0.02792   | 0.03065   | 2.624     | 1.161e+03 | 17.99     |
| 71        | 0.00705   | 0.01792   | 5.373     | 1.16e+03  | 17.14     |
| 72        | 0.008191  | 0.05459   | 3.848     | 1.788e+03 | 10.62     |
| 73        | 0.02599   | 0.09489   | 2.679     | 1.162e+03 | 18.02     |
| 74        | 0.005775  | 0.05029   | 4.3       | 1.404e+03 | 18.87     |
| 75        | 0.01792   | 0.007104  | 3.152     | 1.228e+03 | 21.45     |
| 76        | 0.011     | 0.007914  | 3.428     | 4.561e+03 | 48.25     |
| 77        | 0.008213  | 0.0827    | 3.732     | 1.163e+03 | 19.87     |
| 78        | 0.02604   | 0.06763   | 2.064     | 1.23e+03  | 22.33     |
| 79        | 0.02616   | 0.09027   | 2.966     | 1.161e+03 | 19.79     |
| 80        | 0.0247    | 0.08974   | 

In [30]:
params_best['max_depth'] = int(params_best['max_depth'])
params_best['n_estimators'] = int(params_best['n_estimators'])
params_best['num_leaves'] = int(params_best['num_leaves'])
lgb_reg= lgb.LGBMRegressor(**params_best )
lgb_reg.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000179 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 63
[LightGBM] [Info] Number of data points in the train set: 2415, number of used features: 6
[LightGBM] [Info] Start training from score 0.301664


LGBMRegressor(learning_rate=0.0011, max_depth=3, n_estimators=1404,
              num_leaves=20)

In [31]:
y_pred= lgb_reg.predict(X_test)
y_pred1=lgb_reg.predict(X_val)
y_pred2=lgb_reg.predict(X_train)

In [32]:
print("mean_absolute_error:", mean_absolute_error(y_test, y_pred))
print("mean_squared_error:", mean_squared_error(y_test, y_pred))
print("rmse:", sqrt(mean_squared_error(y_test, y_pred)))
print("r2 score:", r2_score(y_test, y_pred))

mean_absolute_error: 0.06373161430356314
mean_squared_error: 0.006208845138778282
rmse: 0.07879622540945906
r2 score: 0.8788060762581171


In [33]:
print("mean_absolute_error:", mean_absolute_error(y_val, y_pred1))
print("mean_squared_error:", mean_squared_error(y_val, y_pred1))
print("rmse:", sqrt(mean_squared_error(y_val, y_pred1)))
print("r2 score:", r2_score(y_val, y_pred1))

mean_absolute_error: 0.0633796025111148
mean_squared_error: 0.006342511518935127
rmse: 0.07963988648243496
r2 score: 0.8727905554580779


In [34]:
print("mean_absolute_error:", mean_absolute_error(y_train, y_pred2))
print("mean_squared_error:", mean_squared_error(y_train, y_pred2))
print("rmse:", sqrt(mean_squared_error(y_train, y_pred2)))
print("r2 score:", r2_score(y_train, y_pred2))

mean_absolute_error: 0.06130360410020308
mean_squared_error: 0.005719867589499725
rmse: 0.0756298062241318
r2 score: 0.8830658681107654
